# CH 8-9

## TOC<a id='toc'></a>
* [Ch8 Notes](#ch8_notes)
* [Ch9 Notes](#ch9_notes)

### CH8 Notes <a id='ch8_notes'></a>
[toc](#toc)

## Variables are not boxes
* think of variables as labels to objects - more like sticky notes than boxes
    - basically they are all references
* Every object has an identity, a type, and a value
    - identity never changes once it is created
    - the `is` operator compares identity of two objects
    - the `id()` function returns an integer representing the identiy
    - in CPython, `id()` returns the memory address of object (but this is implementation specific)
* in contrast `==` is used to compare the values of objects
    - `a == b` is syntactic sugar for `a.__eq__(b)`
    - it can be overloaded, and complicated
    - the one inherited from `object` compares ids
    - most builtins override `__eq__`
    - `is` operator cannot be overloaded
        * this makes it faster because python does not have to find and invoke special methods, and it basically integer comparison
        * so if it is what is needed, use it over ==
    - if comparing variable to singleton, better to use `is`
        * ex: `x is None`

## Deep vs Shallow
* tuples immutable in a shallow sense - the references theire cannot change. But the object they refer to can, if they are mutable.
* copies are shallow by default
* easiest way to copy most built-in mutable collections is to use the built-in constructor for the type itself
    - `l2 = list(l1)`
    - for list, shortcut `l1 = l2[:]`
    - this produces shallow copy
* `copy` module provides `copy()` and `deepcopy()`
* making a deep copy is not a simple matter in the general case. Objects may have cyclical dependencies.
    - `deepcopy` function remembers the objects already copied to handle cyclica references gracefully.
    - you can control the behavior of copy and deepcopy by implementing `__copy__` and `__deepcopy__` special methods

In [15]:
a = [10, 20]
b = [a, 30]
a.append(b)

In [16]:
a

[10, 20, [[...], 30]]

In [17]:
b

[[10, 20, [...]], 30]

In [18]:
from copy import deepcopy

In [19]:
c = deepcopy(a)

In [20]:
c

[10, 20, [[...], 30]]

In [23]:
id(a), id(a[2]), id(a[2][0])

(1441717222024, 1441699697288, 1441717222024)

In [27]:
id(b), id(b[0]), id(b[0][2])

(1441699697288, 1441717222024, 1441699697288)

In [29]:
id(c), id(c[2]), id(c[2][0])

(1441717202824, 1441717222408, 1441717202824)

## Function parameters
* python is a **pass by sharing** language
    - means each formal parameter of the function gets a copy of each reference in the arguments (i.e. parameters inside function become aliases for the actual arguments)
        - same in most OO languages
    - implies function may change any mutable objects passed as a parameter, but it cannot change the identity of those objects (i.e. cannot replace the object with another)

In [30]:
def f1(a,b):
    a += b
    return a

def f2(a,b):
    a = a + b
    return a

In [32]:
x = [1,2]
y  = [3,4]
f1(x,y)

[1, 2, 3, 4]

In [33]:
x

[1, 2, 3, 4]

In [34]:
x = [1,2]
y  = [3,4]
f2(x,y)

[1, 2, 3, 4]

In [35]:
x

[1, 2]

<br>
<hr>
<font color=blue> Note difference between `__add__` and `__iadd__` </font>
<br>
<hr>

**Don't use mutable types as parameter defaults!**
* default is stored with function, and chages reflected accross different uses of function.
* each default parameter is evaluated when the function is defined (usually at module load)
    - look into `<myFunc>.__defaults__`

* also consider carefully how to handle mutating arguments. Use **the principle of least astonishment**
    - (I think you usually add an underscore at the end of method name to indicate that it mutates parms)

## del and garbage collection
* `del` deletes names, not objects
* Garbage collector deletes objects when there are not more references pointing to object
    - or if object is unreachable (as the case in reference cycles)
    - in CPython, the primary algorithm for garbage collection is refernce counting
* `__del__` does not cause disposal of instance. Rather it is a special method the python interpreter calls when the instance is about to be destroyed.
    - typically used to release external resources.

## Weak References
* a weak reference to an object is a reference that does not increase its reference count
    - thus it does not prevent it from being garbage collected
* careful with unexpected references, like `_` in console sessions, temp vars in loops, and tracebacks
* can use as `wref =  weakref.ref(myobj)` and then call it as `wref()` to return object or None if it is gone
* generally use of weakref.ref is discouraged (too lowe level and complicated) in favor of weakreft collections and finalize.
    * `WeakValueDictionary` holds weakrefs as values - key deleted when value gets garbage collected
    * `WewakKeyDictionary` holdsa weakrefs as keys - can be used to associate additional data to object
    * `WeakSet`
* limitations of weak references
    - plain list and dict may not be referrents; sets and user defined can be
    - can trivial subclass list, and then it can be a referrent
    - most limitations result from optimizations done in the implementation of CPython
    - one weird exmaple of this:

In [1]:
fs = frozenset([1,2,3])
fs2 = fs.copy()
fs is fs2

True

In [2]:
s1 = 'ABC'
s2 = 'ABC'
s1 is s2

True

The above ius an example of **interning** - cpython optimization

### CH9 Notes <a id='ch9_notes'></a>
[toc](#toc)